### Importy
Wykorzystuje następujące dodatkowe biblioteki:
* numpy - generowanie punktów i manipulacja danymi
* scipy - przykładowa implementacja biblioteczna obliczania wyznacznika
* pandas - przechowywanie danych i manipulacja nimi
* seaborn - wizualizacja statystyk

In [1]:
from IPython.display import Markdown
import itertools

import matplotlib.pyplot as plt
import scipy
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib widget
sns.set_theme()
sns_style={"edgecolor":"none", "s": 1}

PLOT_COL_WRAP = 4
PLOT_FIG_W = 20
PLOT_FIG_H = 4

### Implementacja wyznaczników
* det_lib_2x2 - Biblioteczny 2x2
* det_my_2x2 - Własna implementacja 2x2
* det_lib_3x3 - Biblioteczna 3x3
* det_my_3x3 - Własna implementacja 3x3

In [2]:
def det_lib_2x2(df, b, c):
    def det(a):
        return scipy.linalg.det(np.array([[a[0] - c[0], a[1] - c[1]], 
                                       [b[0] - c[0], b[1] - c[1]]]), check_finite=False)
    return df[["x", "y"]].apply(det, axis=1)

def det_my_2x2(df, b, c):
    m00 = df.x - c[0]
    m10 = df.y - c[1]
    m01 = b[0] - c[0]
    m11 = b[1] - c[1]
    return m00 * m11 - m10 * m01

def det_lib_3x3(df, b, c):
    def det(a):
        return scipy.linalg.det(np.array([[a[0], a[1], 1], 
                                       [b[0], b[1], 1], 
                                       [c[0], c[1], 1]]), check_finite=False)
    return df[["x", "y"]].apply(det, axis=1)

def det_my_3x3(df, b, c):
    return df.x * b[1] + \
           df.y * c[0] + \
           b[0] * c[1] - \
           b[1] * c[0] - \
           c[1] * df.x - \
           b[0] * df.y


DETS = {
    "Implementacja biblioteczna 2x2": det_lib_2x2,
    "Implementacja własna 2x2": det_my_2x2,
    "Implementacja biblioteczna 3x3": det_lib_3x3,
    "Implementacja własna 3x3": det_my_3x3,
}
REF_DET = "Implementacja własna 3x3"
TYPES = {
    "float16": np.float16, 
    "float32": np.float32, 
    "float64": np.float64,
    "float128": np.float128,
}

Analizowane typy danych:
* float16 
* float32
* float64
* float128

In [3]:
def df_split_types(points_coord):
    points = pd.DataFrame(index= np.arange(len(points_coord)),
                          columns=pd.MultiIndex.from_product([TYPES.keys(), ["x", "y", *DETS.keys()]]))
    
    for dtype in TYPES.keys():
        print(dtype)
        points.loc(axis=1)[dtype, ["x", "y"]] = points_coord
    
    points = points.astype({ ((ty, col)): ty for ty, col in itertools.product(TYPES.keys(), ["x", "y", *DETS.keys()]) })
    
    return points

### Generowanie $10^5$ losowych punktów o współrzędnych z przedziału $[-1000, 1000]$

In [4]:
rng = np.random.default_rng(420)

points = df_split_types(rng.uniform(-1000, 1000, size=(10 ** 5, 2)))
points.info()
points

float16
float32
float64
float128
<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 24 columns):
 #   Column                                      Non-Null Count   Dtype   
---  ------                                      --------------   -----   
 0   (float16, x)                                100000 non-null  float16 
 1   (float16, y)                                100000 non-null  float16 
 2   (float16, Implementacja biblioteczna 2x2)   0 non-null       float16 
 3   (float16, Implementacja własna 2x2)         0 non-null       float16 
 4   (float16, Implementacja biblioteczna 3x3)   0 non-null       float16 
 5   (float16, Implementacja własna 3x3)         0 non-null       float16 
 6   (float32, x)                                100000 non-null  float32 
 7   (float32, y)                                100000 non-null  float32 
 8   (float32, Implementacja biblioteczna 2x2)   0 non-null       float32 
 9   (float32, Implementacja wła

float16                                             \
               x           y Implementacja biblioteczna 2x2   
0      555.00000  889.500000                            NaN   
1      978.50000 -178.250000                            NaN   
2      289.25000  504.750000                            NaN   
3     -961.50000   -4.796875                            NaN   
4      231.62500 -818.000000                            NaN   
...          ...         ...                            ...   
99995 -323.50000 -730.000000                            NaN   
99996  -60.53125  308.750000                            NaN   
99997 -468.00000 -997.500000                            NaN   
99998 -141.50000 -836.500000                            NaN   
99999 -554.00000 -746.500000                            NaN   

                                                               \
      Implementacja własna 2x2 Implementacja biblioteczna 3x3   
0                          NaN                            NaN   
1                          NaN                            NaN   
2                          NaN                            NaN   
3                          NaN                            NaN   
4                          NaN                            NaN   
...                        ...                            ...   
99995                      NaN                            NaN   
99996                      NaN                            NaN   
99997                      NaN                            NaN   
99998                      NaN                            NaN   
99999                      NaN                            NaN   

                                   float32              \
      Implementacja własna 3x3           x           y   
0                          NaN  554.853516  889.466919   
1                          NaN  978.583313 -178.228439   
2                          NaN  289.193726  504.652008   
3                          NaN -961.615051   -4.798686   
4                          NaN  231.628983 -818.011108   
...                        ...         ...         ...   
99995                      NaN -323.615387 -729.782104   
99996                      NaN  -60.516380  308.793152   
99997                      NaN -468.107788 -997.720825   
99998                      NaN -141.505112 -836.424988   
99999                      NaN -553.881104 -746.293884   

                                                               ...  \
      Implementacja biblioteczna 2x2 Implementacja własna 2x2  ...   
0                                NaN                      NaN  ...   
1                                NaN                      NaN  ...   
2                                NaN                      NaN  ...   
3                                NaN                      NaN  ...   
4                                NaN                      NaN  ...   
...                              ...                      ...  ...   
99995                            NaN                      NaN  ...   
99996                            NaN                      NaN  ...   
99997                            NaN                      NaN  ...   
99998                            NaN                      NaN  ...   
99999                            NaN                      NaN  ...   

                             float64                           \
      Implementacja biblioteczna 2x2 Implementacja własna 2x2   
0                                NaN                      NaN   
1                                NaN                      NaN   
2                                NaN                      NaN   
3                                NaN                      NaN   
4                                NaN                      NaN   
...                              ...                      ...   
99995                            NaN                      NaN   
99996                            NaN                      NaN   
99997                      

### Przeliczanie wartości wyznaczników

In [5]:
def calc_det(points):
    for dtype in TYPES:
        for name, det in DETS.items():
            print(dtype, name)
            points.loc[:, (dtype, name)] = det(points.loc(axis=1)[dtype], (-1.0, 0.0), (1.0, 0.1)).astype(TYPES[dtype])
calc_det(points)
points.info()
points

float16 Implementacja biblioteczna 2x2
float16 Implementacja własna 2x2
float16 Implementacja biblioteczna 3x3
float16 Implementacja własna 3x3
float32 Implementacja biblioteczna 2x2
float32 Implementacja własna 2x2
float32 Implementacja biblioteczna 3x3
float32 Implementacja własna 3x3
float64 Implementacja biblioteczna 2x2
float64 Implementacja własna 2x2
float64 Implementacja biblioteczna 3x3
float64 Implementacja własna 3x3
float128 Implementacja biblioteczna 2x2
float128 Implementacja własna 2x2
float128 Implementacja biblioteczna 3x3
float128 Implementacja własna 3x3
<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 24 columns):
 #   Column                                      Non-Null Count   Dtype   
---  ------                                      --------------   -----   
 0   (float16, x)                                100000 non-null  float16 
 1   (float16, y)                                100000 non-null  float16 
 2   (floa

float16                                             \
               x           y Implementacja biblioteczna 2x2   
0      555.00000  889.500000                      1723.0000   
1      978.50000 -178.250000                      -454.5000   
2      289.25000  504.750000                       980.5000   
3     -961.50000   -4.796875                        86.4375   
4      231.62500 -818.000000                     -1659.0000   
...          ...         ...                            ...   
99995 -323.50000 -730.000000                     -1428.0000   
99996  -60.53125  308.750000                       623.5000   
99997 -468.00000 -997.500000                     -1948.0000   
99998 -141.50000 -836.500000                     -1659.0000   
99999 -554.00000 -746.500000                     -1438.0000   

                                                               \
      Implementacja własna 2x2 Implementacja biblioteczna 3x3   
0                    1724.0000                      1723.0000   
1                    -454.5000                      -454.5000   
2                     980.5000                       980.5000   
3                      86.4375                        86.4375   
4                   -1659.0000                     -1659.0000   
...                        ...                            ...   
99995               -1428.0000                     -1428.0000   
99996                 623.5000                       623.5000   
99997               -1948.0000                     -1948.0000   
99998               -1659.0000                     -1659.0000   
99999               -1438.0000                     -1438.0000   

                                   float32              \
      Implementacja własna 3x3           x           y   
0                    1724.0000  554.853516  889.466919   
1                    -454.5000  978.583313 -178.228439   
2                     980.5000  289.193726  504.652008   
3                      86.4375 -961.615051   -4.798686   
4                   -1659.0000  231.628983 -818.011108   
...                        ...         ...         ...   
99995               -1428.0000 -323.615387 -729.782104   
99996                 623.5000  -60.516380  308.793152   
99997               -1948.0000 -468.107788 -997.720825   
99998               -1659.0000 -141.505112 -836.424988   
99999               -1438.0000 -553.881104 -746.293884   

                                                               ...  \
      Implementacja biblioteczna 2x2 Implementacja własna 2x2  ...   
0                        1723.348511              1723.348511  ...   
1                        -454.415222              -454.415222  ...   
2                         980.284668               980.284668  ...   
3                          86.464134                86.464134  ...   
4                       -1659.285156             -1659.285156  ...   
...                              ...                      ...  ...   
99995                   -1427.302612             -1427.302612  ...   
99996                     623.537964               623.537964  ...   
99997                   -1948.730835             -1948.730835  ...   
99998                   -1658.799438             -1658.799438  ...   
99999                   -1437.299683             -1437.299683  ...   

                             float64                           \
      Implementacja biblioteczna 2x2 Implementacja własna 2x2   
0                        1723.348466              1723.348466   
1                        -454.415213              -454.415213   
2                         980.284616               980.284616   
3                          86.464134                86.464134   
4                       -1659.285167             -1659.285167   
...                              ...                      ...   
99995                   -1427.302634             -1427.302634   
99996                     623.537924               623.537924   
99997                   -19

### Funkcja rysująca punkty

In [6]:
def plot(points, dtype, det, epsilon, ax):
    df = points.loc(axis=1)[dtype]
    sns.scatterplot(data=df[df.loc[:, det] >  epsilon], x="x", y="y", color="red", ax=ax, **sns_style)
    sns.scatterplot(data=df[df.loc[:, det] < -epsilon], x="x", y="y", color="blue", ax=ax, **sns_style)
    sns.scatterplot(data=df[df.loc[:, det].abs() <= epsilon], x="x", y="y", color="green", ax=ax, **sns_style)

def plot_types_sides(name, points):
    display(Markdown("### Funkcja rysująca punkty"))
#     fig, ax = plt.subplots(2, 2)
    fig, ax = plt.subplots(1, 4)
    fig.set_figwidth(PLOT_FIG_W)
    fig.set_figheight(PLOT_FIG_H)
    for i, dtype in enumerate(TYPES):
#         a = ax[i // 2][i % 2]
        a = ax[i]
        a.set_title(dtype)
        plot(points, dtype, REF_DET, 1e1, ax=a)
    
    fig.savefig(f"Lab1Raport/{name}.png")
        
    plt.show()

plot_types_sides("tmp", points)

### Funkcja rysująca punkty

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Zestawienie klasyfikacji punktów jako leżące na prostej względem $\epsilon$

In [7]:
epsilons = 10 ** (np.arange(-16, 0, 1.0))

def classificate_points(points):
    display(Markdown("### Zestawienie klasyfikacji punktów jako leżące na prostej względem $\epsilon$"))
    epsilon_df = pd.DataFrame(index=epsilons, columns=pd.MultiIndex.from_product([DETS.keys(), TYPES.keys()]), dtype=np.int64)

    for e in epsilon_df.index:
        epsilon_df.at[e] = (points.loc(axis=1)[:, DETS.keys()].abs() <= e).sum().swaplevel(0, 1)

    display(epsilon_df)

    epsilon_df_pivot = epsilon_df.stack().reset_index(col_fill="a")
    epsilon_df_pivot = epsilon_df_pivot.rename(columns={
        epsilon_df_pivot.columns[0]: "epsilon",
        epsilon_df_pivot.columns[1]: "dtype",
    })
    epsilon_df_pivot = epsilon_df_pivot.melt(id_vars=["epsilon", "dtype"], 
                                             var_name="det", 
                                             value_name="n")
    
    return epsilon_df, epsilon_df_pivot

epsilon_df, epsilon_df_pivot = classificate_points(points)

### Zestawienie klasyfikacji punktów jako leżące na prostej względem $\epsilon$

Implementacja biblioteczna 2x2                           \
                                    float16 float32 float64 float128   
1.000000e-16                            0.0     0.0     0.0      0.0   
1.000000e-15                            0.0     0.0     0.0      0.0   
1.000000e-14                            0.0     0.0     0.0      0.0   
1.000000e-13                            0.0     0.0     0.0      0.0   
1.000000e-12                            0.0     0.0     0.0      0.0   
1.000000e-11                            0.0     0.0     0.0      0.0   
1.000000e-10                            0.0     0.0     0.0      0.0   
1.000000e-09                            0.0     0.0     0.0      0.0   
1.000000e-08                            0.0     0.0     0.0      0.0   
1.000000e-07                            0.0     0.0     0.0      0.0   
1.000000e-06                            0.0     0.0     0.0      0.0   
1.000000e-05                            0.0     0.0     0.0      0.0   
1.000000e-04                            0.0     0.0     0.0      0.0   
1.000000e-03                            0.0     0.0     0.0      0.0   
1.000000e-02                            0.0     0.0     0.0      0.0   
1.000000e-01                            6.0     6.0     6.0      6.0   

             Implementacja własna 2x2                           \
                              float16 float32 float64 float128   
1.000000e-16                      1.0     0.0     0.0      0.0   
1.000000e-15                      1.0     0.0     0.0      0.0   
1.000000e-14                      1.0     0.0     0.0      0.0   
1.000000e-13                      1.0     0.0     0.0      0.0   
1.000000e-12                      1.0     0.0     0.0      0.0   
1.000000e-11                      1.0     0.0     0.0      0.0   
1.000000e-10                      1.0     0.0     0.0      0.0   
1.000000e-09                      1.0     0.0     0.0      0.0   
1.000000e-08                      1.0     0.0     0.0      0.0   
1.000000e-07                      1.0     0.0     0.0      0.0   
1.000000e-06                      1.0     0.0     0.0      0.0   
1.000000e-05                      1.0     0.0     0.0      0.0   
1.000000e-04                      1.0     0.0     0.0      0.0   
1.000000e-03                      1.0     0.0     0.0      0.0   
1.000000e-02                      1.0     0.0     0.0      0.0   
1.000000e-01                      5.0     6.0     6.0      6.0   

             Implementacja biblioteczna 3x3                           \
                                    float16 float32 float64 float128   
1.000000e-16                            0.0     0.0     0.0      0.0   
1.000000e-15                            0.0     0.0     0.0      0.0   
1.000000e-14                            0.0     0.0     0.0      0.0   
1.000000e-13                            0.0     0.0     0.0      0.0   
1.000000e-12                            0.0     0.0     0.0      0.0   
1.000000e-11                            0.0     0.0     0.0      0.0   
1.000000e-10                            0.0     0.0     0.0      0.0   
1.000000e-09                            0.0     0.0     0.0      0.0   
1.000000e-08                            0.0     0.0     0.0      0.0   
1.000000e-07                            0.0     0.0     0.0      0.0   
1.000000e-06                            0.0     0.0     0.0      0.0   
1.000000e-05                            0.0     0.0     0.0      0.0   
1.000000e-04                            0.0     0.0     0.0      0.0   
1.000000e-03                            0.0     0.0     0.0      0.0   
1.000000e-02                            0.0     0.0     0.0      0.0   
1.000000e-01                            6.0     6.0     6.0      6.0   

             Implementacja własna 3x3                           
                              float16 float32 float64 float128  
1.000000e-16                      0.0     0.0     0.0      0.0  
1.000000e-15                      0

### Zestawnie liczby punktów zakwalifikowanych za leżące na prostej

In [8]:
def plot_epsilon(name, epsilon_df_pivot):
    display(Markdown("### Zestawnie liczby punktów zakwalifikowanych za leżące na prostej"))
    
    g = sns.FacetGrid(epsilon_df_pivot, col="dtype", hue="det", col_order=TYPES.keys(), col_wrap=PLOT_COL_WRAP)
    g.map(sns.lineplot, "epsilon", "n")
    g.set(xscale="log")
    g.add_legend(bbox_to_anchor=(0.5, 0.5, 0.445, 0))
#     g.add_legend(loc="lower center")
    g.fig.set_figwidth(PLOT_FIG_W)
    g.fig.set_figheight(PLOT_FIG_H)
    g.fig.savefig(f"Lab1Raport/{name} - epsilon n type.png")


    g = sns.FacetGrid(epsilon_df_pivot, col="det", hue="dtype", col_wrap=PLOT_COL_WRAP)
    g.map(sns.lineplot, "epsilon", "n")
    g.set(xscale="log")
    g.add_legend(loc="center right")
    g.fig.set_figwidth(PLOT_FIG_W)
    g.fig.set_figheight(PLOT_FIG_H)
    g.fig.savefig(f"Lab1Raport/{name} - epsilon n det.png")

plot_epsilon("tmp", epsilon_df_pivot)

### Zestawnie liczby punktów zakwalifikowanych za leżące na prostej

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Porównanie różnic klasyfikacji punktów za leżące na prostej względem implementacji wyznacznika oraz typu

In [9]:
def plot_diffs(name, epsilon_df):
    display(Markdown("### Porównanie różnic klasyfikacji punktów za leżące na prostej względem implementacji wyznacznika oraz typu"))
    diffs_det = pd.DataFrame({"epsilon": epsilons}, columns=["epsilon", *DETS.keys()])
    for det in DETS.keys():
        diffs_det[det] =  (epsilon_df.loc(axis=1)[det, :].max(axis=1) - epsilon_df.loc(axis=1)[det, :].min(axis=1)).reset_index(drop=True)

    diffs_det = diffs_det.melt(id_vars=["epsilon"], 
                               var_name="det", 
                               value_name="diff")

    g = sns.FacetGrid(data=diffs_det, col="det", col_wrap=PLOT_COL_WRAP)
    g.map(sns.lineplot, "epsilon", "diff")
    g.set(xscale="log")
    g.fig.set_figwidth(PLOT_FIG_W)
    g.fig.set_figheight(PLOT_FIG_H)
    g.add_legend(loc="center right")
    g.fig.savefig(f"Lab1Raport/{name} - epsilon diff type.png")



    diffs_types = pd.DataFrame({"epsilon": epsilons}, columns=["epsilon", *TYPES.keys()])
    for ty in TYPES.keys():
        diffs_types[ty] =  (epsilon_df.loc(axis=1)[:, ty].max(axis=1) - epsilon_df.loc(axis=1)[:, ty].min(axis=1)).reset_index(drop=True)

    diffs_types = diffs_types.melt(id_vars=["epsilon"], 
                               var_name="type", 
                               value_name="diff")

#     g = sns.FacetGrid(data=diffs_types, hue="type", col_wrap=PLOT_COL_WRAP)
#     g.map(sns.lineplot, "epsilon", "diff")
    fig, ax = plt.subplots()
    g = sns.lineplot(data=diffs_types, x="epsilon", y="diff", hue="type", ax=ax)
    g.set(xscale="log")

    fig.set_figwidth(PLOT_FIG_W / 3.0)
    fig.set_figheight(PLOT_FIG_W / 3.0)
#     g.add_legend(bbox_to_anchor=(0.5, 0.5, 0.4, 0))
    fig.savefig(f"Lab1Raport/{name} - epsilon diff det.png")
    
#     g.fig.set_figwidth(PLOT_FIG_W)
#     g.fig.set_figheight(PLOT_FIG_H)
#     g.add_legend(bbox_to_anchor=(0.5, 0.5, 0.4, 0))
#     g.fig.savefig(f"Lab1Raport/{name} - epsilon diff det.png")

plot_diffs("tmp", epsilon_df)

### Porównanie różnic klasyfikacji punktów za leżące na prostej względem implementacji wyznacznika oraz typu

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
def full_raport(fname, points):
    points = df_split_types(points)
    calc_det(points)
        
    display(Markdown("### Punkty oraz wyloczone wartości wyznaczników za pomocą różnych implementacji oraz reprezentowane różnymi reprezentacjami liczb zmienno przecinkowych"))
    display(points) 

    plot_types_sides(fname, points)
    epsilon_df, epsilon_df_pivot = classificate_points(points)
    
    dets = list(DETS.keys())
    for det in [[dets[0], dets[1]], [dets[2], dets[3]]]:
        name = " & ".join(det)
        tmp = epsilon_df.loc(axis=1)[det].copy()
        tmp.index = np.log10(tmp.index).map(lambda e: f"$ 10^{{{e}}} $")
        tmp = tmp.astype(np.int)
    
        tex = tmp.to_latex(escape=False, caption=f"{fname} - {name}".replace("&", "\&"))
        tex = tex.replace("\\begin{table}", "\\begin{table}[H]")
        tex = tex.replace("{lrrrrrrrr}", "{l|rrrr|rrrr}")

        with open(f"Lab1Raport/{fname} - {name}.tex", "w") as f:
            f.write(tex)
            f.close()

    plot_epsilon(fname, epsilon_df_pivot)
    plot_diffs(fname, epsilon_df)
    

In [11]:
rng = np.random.default_rng(42)
full_raport("Zestaw a", rng.uniform(-1000, 1000, size=(10 ** 5, 2)))

float16
float32
float64
float128
float16 Implementacja biblioteczna 2x2
float16 Implementacja własna 2x2
float16 Implementacja biblioteczna 3x3
float16 Implementacja własna 3x3
float32 Implementacja biblioteczna 2x2
float32 Implementacja własna 2x2
float32 Implementacja biblioteczna 3x3
float32 Implementacja własna 3x3
float64 Implementacja biblioteczna 2x2
float64 Implementacja własna 2x2
float64 Implementacja biblioteczna 3x3
float64 Implementacja własna 3x3
float128 Implementacja biblioteczna 2x2
float128 Implementacja własna 2x2
float128 Implementacja biblioteczna 3x3
float128 Implementacja własna 3x3


### Punkty oraz wyloczone wartości wyznaczników za pomocą różnych implementacji oraz reprezentowane różnymi reprezentacjami liczb zmienno przecinkowych

float16                                            \
             x          y Implementacja biblioteczna 2x2   
0      548.000 -122.25000                     -299.50000   
1      717.000  394.75000                      717.50000   
2     -811.500  951.00000                     1983.00000   
3      522.500  572.00000                     1092.00000   
4     -744.000  -99.25000                     -124.18750   
...        ...        ...                            ...   
99995  111.125  -26.03125                      -63.28125   
99996  432.250 -802.50000                    -1648.00000   
99997  270.750 -638.50000                    -1304.00000   
99998  242.500 -602.50000                    -1229.00000   
99999 -781.000 -914.50000                    -1751.00000   

                                                               \
      Implementacja własna 2x2 Implementacja biblioteczna 3x3   
0                      -299.50                     -299.50000   
1                       718.00                      717.50000   
2                      1983.00                     1983.00000   
3                      1092.00                     1092.00000   
4                      -124.25                     -124.18750   
...                        ...                            ...   
99995                   -63.25                      -63.28125   
99996                 -1648.00                    -1648.00000   
99997                 -1304.00                    -1304.00000   
99998                 -1229.00                    -1229.00000   
99999                 -1751.00                    -1751.00000   

                                   float32              \
      Implementacja własna 3x3           x           y   
0                   -299.50000  547.912109 -122.243118   
1                    718.00000  717.195862  394.736053   
2                   1983.00000 -811.645325  951.244690   
3                   1092.00000  522.279419  572.128601   
4                   -124.25000 -743.772705  -99.228127   
...                        ...         ...         ...   
99995                -63.28125  111.109215  -26.032192   
99996              -1648.00000  432.337219 -802.728210   
99997              -1304.00000  270.781799 -638.433655   
99998              -1230.00000  242.468018 -602.713135   
99999              -1751.00000 -780.950989 -914.540588   

                                                               ...  \
      Implementacja biblioteczna 2x2 Implementacja własna 2x2  ...   
0                        -299.377441              -299.377441  ...   
1                         717.652527               717.652527  ...   
2                        1983.553955              1983.553955  ...   
3                        1091.929199              1091.929199  ...   
4                        -124.178986              -124.178986  ...   
...                              ...                      ...  ...   
99995                     -63.275307               -63.275307  ...   
99996                   -1648.790161             -1648.790161  ...   
99997                   -1304.045532             -1304.045532  ...   
99998                   -1229.773071             -1229.773071  ...   
99999                   -1751.086060             -1751.086060  ...   

                             float64                           \
      Implementacja biblioteczna 2x2 Implementacja własna 2x2   
0                        -299.377451              -299.377451   
1                         717.652532               717.652532   
2                        1983.553937              1983.553937   
3                        1091.929281              1091.929281   
4                        -124.178975              -124.178975   
...                              ...                      ...   
99995                     -63.275306               -63.275306   
99996                   -1648.790101             -1648.790101   
99997                   -1304.045518             -1304.045518   

### Funkcja rysująca punkty

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Porównanie różnic klasyfikacji punktów za leżące na prostej względem implementacji wyznacznika oraz typu

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
rng = np.random.default_rng(42)
full_raport("Zestaw b", rng.uniform(-1e14, 1e14, size=(10 ** 5, 2)))

float16
float32
float64
float128
float16 Implementacja biblioteczna 2x2
float16 Implementacja własna 2x2
float16 Implementacja biblioteczna 3x3
float16 Implementacja własna 3x3
float32 Implementacja biblioteczna 2x2
float32 Implementacja własna 2x2
float32 Implementacja biblioteczna 3x3
float32 Implementacja własna 3x3
float64 Implementacja biblioteczna 2x2
float64 Implementacja własna 2x2
float64 Implementacja biblioteczna 3x3
float64 Implementacja własna 3x3
float128 Implementacja biblioteczna 2x2
float128 Implementacja własna 2x2
float128 Implementacja biblioteczna 3x3
float128 Implementacja własna 3x3


### Punkty oraz wyloczone wartości wyznaczników za pomocą różnych implementacji oraz reprezentowane różnymi reprezentacjami liczb zmienno przecinkowych

float16                                                               \
            x    y Implementacja biblioteczna 2x2 Implementacja własna 2x2   
0         inf -inf                           -inf                     -inf   
1         inf  inf                           -inf                      NaN   
2        -inf  inf                            inf                      inf   
3         inf  inf                           -inf                      NaN   
4        -inf -inf                            inf                      NaN   
...       ...  ...                            ...                      ...   
99995     inf -inf                           -inf                     -inf   
99996     inf -inf                           -inf                     -inf   
99997     inf -inf                           -inf                     -inf   
99998     inf -inf                           -inf                     -inf   
99999    -inf -inf                            inf                      NaN   

                                                                    float32  \
      Implementacja biblioteczna 3x3 Implementacja własna 3x3             x   
0                               -inf                      NaN  5.479121e+13   
1                               -inf                      NaN  7.171959e+13   
2                                inf                      NaN -8.116453e+13   
3                               -inf                      NaN  5.222794e+13   
4                                inf                      NaN -7.437727e+13   
...                              ...                      ...           ...   
99995                           -inf                      NaN  1.111092e+13   
99996                           -inf                      NaN  4.323372e+13   
99997                           -inf                      NaN  2.707818e+13   
99998                           -inf                      NaN  2.424680e+13   
99999                            inf                      NaN -7.809510e+13   

                                                                             \
                  y Implementacja biblioteczna 2x2 Implementacja własna 2x2   
0     -1.222431e+13                  -2.992774e+13            -2.992774e+13   
1      3.947361e+13                   7.177525e+13             7.177525e+13   
2      9.512447e+13                   1.983654e+14             1.983654e+14   
3      5.721286e+13                   1.092029e+14             1.092029e+14   
4     -9.922812e+12                  -1.240790e+13            -1.240790e+13   
...             ...                            ...                      ...   
99995 -2.603219e+12                  -6.317530e+12            -6.317530e+12   
99996 -8.027282e+13                  -1.648690e+14            -1.648690e+14   
99997 -6.384337e+13                  -1.303946e+14            -1.303946e+14   
99998 -6.027131e+13                  -1.229673e+14            -1.229673e+14   
99999 -9.145406e+13                  -1.750986e+14            -1.750986e+14   

       ...                        float64                           \
       ... Implementacja biblioteczna 2x2 Implementacja własna 2x2   
0      ...                  -2.992775e+13            -2.992775e+13   
1      ...                   7.177525e+13             7.177525e+13   
2      ...                   1.983654e+14             1.983654e+14   
3      ...                   1.092029e+14             1.092029e+14   
4      ...                  -1.240790e+13            -1.240790e+13   
...    ...                            ...                      ...   
99995  ...                  -6.317531e+12            -6.317531e+12   
99996  ...                  -1.648690e+14            -1.648690e+14   
99997  ...                  -1.303946e+14            -1.303946e+14   
99998  ...                  -1.229673e+14            -1.229673e+14   
99999  ...                  -1.750986e+14            -1.750986e+14   

                        

### Funkcja rysująca punkty

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Porównanie różnic klasyfikacji punktów za leżące na prostej względem implementacji wyznacznika oraz typu

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
rng = np.random.default_rng(42)
alpha = rng.uniform(0, np.pi * 2, size=1000)
full_raport("Zestaw c", np.column_stack((np.cos(alpha), np.sin(alpha))) * 100.0)

float16
float32
float64
float128
float16 Implementacja biblioteczna 2x2
float16 Implementacja własna 2x2
float16 Implementacja biblioteczna 3x3
float16 Implementacja własna 3x3
float32 Implementacja biblioteczna 2x2
float32 Implementacja własna 2x2
float32 Implementacja biblioteczna 3x3
float32 Implementacja własna 3x3
float64 Implementacja biblioteczna 2x2
float64 Implementacja własna 2x2
float64 Implementacja biblioteczna 3x3
float64 Implementacja własna 3x3
float128 Implementacja biblioteczna 2x2
float128 Implementacja własna 2x2
float128 Implementacja biblioteczna 3x3
float128 Implementacja własna 3x3


### Punkty oraz wyloczone wartości wyznaczników za pomocą różnych implementacji oraz reprezentowane różnymi reprezentacjami liczb zmienno przecinkowych

float16                                            \
              x          y Implementacja biblioteczna 2x2   
0     14.992188 -98.875000                    -199.375000   
1    -92.687500  37.468750                      84.125000   
2     63.062500 -77.625000                    -161.625000   
3    -32.468750 -94.562500                    -186.000000   
4     83.000000  55.781250                     103.187500   
..          ...        ...                            ...   
995  -92.500000  37.968750                      85.062500   
996  -64.187500 -76.625000                    -146.875000   
997  100.000000   1.450195                      -7.199219   
998   99.937500  -4.164062                     -18.421875   
999  -19.296875  98.125000                     198.125000   

                                                             \
    Implementacja własna 2x2 Implementacja biblioteczna 3x3   
0                -199.375000                    -199.375000   
1                  84.125000                      84.125000   
2                -161.750000                    -161.625000   
3                -186.000000                    -186.000000   
4                 103.187500                     103.187500   
..                       ...                            ...   
995                85.125000                      85.062500   
996              -147.000000                    -146.875000   
997                -7.195312                      -7.199219   
998               -18.421875                     -18.421875   
999               198.000000                     198.125000   

                                float32             \
    Implementacja własna 3x3          x          y   
0                -199.375000  14.995256 -98.869316   
1                  84.125000 -92.715927  37.466747   
2                -161.750000  63.061150 -77.609871   
3                -186.000000 -32.470181 -94.581642   
4                 103.125000  82.997482  55.780083   
..                       ...        ...        ...   
995                85.125000 -92.505241  37.983952   
996              -147.000000 -64.218689 -76.654808   
997                -7.199219  99.989479   1.450594   
998               -18.421875  99.913208  -4.165490   
999               198.000000 -19.291071  98.121628   

                                                             ...  \
    Implementacja biblioteczna 2x2 Implementacja własna 2x2  ...   
0                      -199.338165              -199.338150  ...   
1                        84.105087                84.105087  ...   
2                      -161.625854              -161.625854  ...   
3                      -186.016266              -186.016266  ...   
4                       103.160416               103.160423  ...   
..                             ...                      ...  ...   
995                      85.118423                85.118431  ...   
996                    -146.987747              -146.987747  ...   
997                      -7.197761                -7.197761  ...   
998                     -18.422300               -18.422300  ...   
999                     198.072357               198.072372  ...   

                           float64                           \
    Implementacja biblioteczna 2x2 Implementacja własna 2x2   
0                      -199.338164              -199.338164   
1                        84.105086                84.105086   
2                      -161.625850              -161.625850   
3                      -186.016270              -186.016270   
4                       103.160414               103.160414   
..                             ...                      ...   
995                      85.118429                85.118429   
996                    -146.987748              -146.987748   
997                      -7.197760                -7.197760   
998                     -18.422300               -18.422300   
999                     198.072370               1

### Funkcja rysująca punkty

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Porównanie różnic klasyfikacji punktów za leżące na prostej względem implementacji wyznacznika oraz typu

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
rng = np.random.default_rng(42)
x = rng.uniform(-1000, 1000, size=1000)
full_raport("Zestaw d", np.column_stack((x, 0.05 * x + 0.05)))

float16
float32
float64
float128
float16 Implementacja biblioteczna 2x2
float16 Implementacja własna 2x2
float16 Implementacja biblioteczna 3x3
float16 Implementacja własna 3x3
float32 Implementacja biblioteczna 2x2
float32 Implementacja własna 2x2
float32 Implementacja biblioteczna 3x3
float32 Implementacja własna 3x3
float64 Implementacja biblioteczna 2x2
float64 Implementacja własna 2x2
float64 Implementacja biblioteczna 3x3
float64 Implementacja własna 3x3
float128 Implementacja biblioteczna 2x2
float128 Implementacja własna 2x2
float128 Implementacja biblioteczna 3x3
float128 Implementacja własna 3x3


### Punkty oraz wyloczone wartości wyznaczników za pomocą różnych implementacji oraz reprezentowane różnymi reprezentacjami liczb zmienno przecinkowych

float16                                            \
          x          y Implementacja biblioteczna 2x2   
0    548.00  27.453125                       0.006248   
1   -122.25  -6.062500                       0.000000   
2    717.00  35.906250                       0.012497   
3    394.75  19.781250                      -0.012497   
4   -811.50 -40.531250                      -0.012497   
..      ...        ...                            ...   
995 -124.00  -6.152344                      -0.004688   
996  278.00  13.953125                       0.006248   
997 -995.50 -49.718750                       0.012497   
998  986.50  49.375000                      -0.000000   
999 -438.25 -21.859375                       0.006248   

                                                             \
    Implementacja własna 2x2 Implementacja biblioteczna 3x3   
0                   0.031250                       0.006248   
1                  -0.007812                       0.000000   
2                   0.062500                       0.012497   
3                   0.000000                      -0.012497   
4                   0.000000                      -0.012497   
..                       ...                            ...   
995                -0.007812                      -0.004688   
996                 0.015625                       0.006248   
997                 0.000000                       0.012497   
998                 0.062500                      -0.000000   
999                 0.000000                       0.006248   

                                 float32             \
    Implementacja własna 3x3           x          y   
0                   0.031250  547.912109  27.445604   
1                  -0.007812 -122.243118  -6.062156   
2                   0.031250  717.195862  35.909794   
3                   0.000000  394.736053  19.786802   
4                  -0.031250 -811.645325 -40.532265   
..                       ...         ...        ...   
995                -0.007812 -124.020798  -6.151040   
996                 0.007812  278.027313  13.951365   
997                -0.031250 -995.382446 -49.719124   
998                 0.031250  986.737000  49.386848   
999                 0.000000 -438.207214 -21.860361   

                                                             ...  \
    Implementacja biblioteczna 2x2 Implementacja własna 2x2  ...   
0                    -2.288818e-06            -3.814697e-06  ...   
1                    -5.722046e-07             0.000000e+00  ...   
2                     1.525879e-06             0.000000e+00  ...   
3                    -7.629395e-07            -3.814697e-06  ...   
4                     3.051758e-06             7.629395e-06  ...   
..                             ...                      ...  ...   
995                   5.722046e-07             9.536743e-07  ...   
996                  -2.288818e-06            -3.814697e-06  ...   
997                  -3.051758e-06             0.000000e+00  ...   
998                  -3.051758e-06             0.000000e+00  ...   
999                  -7.629395e-07             0.000000e+00  ...   

                           float64                           \
    Implementacja biblioteczna 2x2 Implementacja własna 2x2   
0                    -7.589930e-15            -7.105427e-15   
1                     1.710342e-15             0.000000e+00   
2                     0.000000e+00             0.000000e+00   
3                     0.000000e+00             0.000000e+00   
4                    -1.127772e-14             0.000000e+00   
..                             ...                      ...   
995                   0.000000e+00             0.000000e+00   
996                   3.844526e-15             3.552714e-15   
997                   0.000000e+00            -1.421085e-14   
998                   0.000000e+00             0.000000e+00   
999                   0.000000e+00             0.000000e+00   

                     

### Funkcja rysująca punkty

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  # Remove the CWD from sys.path while we load stuff.


...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Porównanie różnic klasyfikacji punktów za leżące na prostej względem implementacji wyznacznika oraz typu

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
print(np.float16, np.finfo(np.float16).nexp, np.finfo(np.float16).nmant)
print(np.float32, np.finfo(np.float32).nexp, np.finfo(np.float32).nmant)
print(np.float64, np.finfo(np.float64).nexp, np.finfo(np.float64).nmant)
print(np.float128, np.finfo(np.float128).nexp, np.finfo(np.float128).nmant)


<class 'numpy.float16'> 5 10
<class 'numpy.float32'> 8 23
<class 'numpy.float64'> 11 52
<class 'numpy.float128'> 15 63
